## JWST F560W LMC Flat Region

Data is setup in a subdirectory that is given in the filter variable below (i.e., "F560W"). Inside this directory, there should be 4 subdirectories called stage0, stage1, stage2, and stage3. The uncal (raw) fits files should be in the stage0 directory.

Author: Karl Gordon

Last modified: 5 Jul 2022

Based on JWebbinar notebooks by Bryan Hilbert, MIRI simulated notebooks by Mattia Libralato, input from various MIRI team members, and input from the jwst pipeline developers.

Uses custom code given in helpers subdirectory.

Extensively tested on MIRI ERO mosaics.

### Variables controling processing and display

In [1]:
filter = "F560W"
fwhm = 1.6
basename = "jw0104000100"

crval = None
rotation = None

# set to a specific rotation if desired
# rotation = 0.

# shifts determine from previous runs with treakreg or with other filters
#   analysis using Analyze_teakreg_shifts.ipynb
#   run this notebook after running shortest wavelength dataset with tweakreg=True and align_to_gaia=True
#     and setting all the tile?_shifts to [0.0, 0.0]
#   use the resulting shifts here and set tweakreg=False and align_to_gaia=False

# initial run
tile1_shifts = [0.0, 0.0]
tile1_shifts = [0.0, 0.0]
tile1_shifts = [0.0, 0.0]
tile1_shifts = [0.0, 0.0]
tile1_shifts = [0.0, 0.0]
tweakreg = True
align_to_gaia = True

# after analysis and for all other filters
# tile1_shifts = [0.10637643165419682, -0.11768223374643519]
# tile2_shifts = [0.10700657405766205, -0.13107961726782408]
# tile3_shifts = [0.10163923831678442, -0.12152158617752193]
# tile4_shifts = [0.09065352553736802, -0.12144904023219313]
# tile5_shifts = [-0.975754312382031, 1.0456499145482434]
# tweakreg = False
# align_to_gaia = False

# only needed if multiple datasets combined that are taken separately (e.g., weeks apart)
matchbkg = False

# display range for uncalibrated images
drange_raw = [3., 20.]
# display range for calibrated images
drange_cal = [1., 6.]
dmap = "afmhot"  # same as ds9 bb

# specify the linearity file to use if default not desired
# linfile = ""
linfile = None

# specify the flat file to use if default not desired
# flatfile = f"RefFiles/DeliveredFlats/MIRI_FM_MIRIMAGE_FASTR1_{filter}_FLAT_09.01.00_kdg_30jun22.fits"
flatfile = None

run_detector1 = True
run_image2 = True
run_image3 = True

In [2]:
import os
import glob
import copy

from astropy.io import ascii
from astropy.visualization import simple_norm

In [3]:
import astropy
print(astropy.__version__)

5.0.4


In [4]:
os.environ["CRDS_PATH"] = "/home/kgordon/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import ImageGrid

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

In [6]:
# List of possible data quality flags
from jwst.datamodels import dqflags
from jwst import datamodels

from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

# custom helper routines
from helpers.jwst_helpers import show_image, overlay_catalog
from helpers.miri_helpers import miri_detector1, miri_image2, miri_image3
from helpers.miri_clean import fix_rateints_to_rate, shift_cal_wcs

In [7]:
import jwst
print(jwst.__version__)

1.5.4.dev17+geec8bb0d


In [8]:
import stcal
print(stcal.__version__)

1.0.1.dev1+g51f5dfc


## CALWEBB_DETECTOR1

In [9]:
miri_uncal_files = glob.glob(f"./{filter}/stage0/*uncal.fits")
output_dir = f'./{filter}/stage1'
print(miri_uncal_files)

['./F560W/stage0/jw01040001001_03101_00001_mirimage_uncal.fits', './F560W/stage0/jw01040001001_03101_00002_mirimage_uncal.fits', './F560W/stage0/jw01040001001_03101_00003_mirimage_uncal.fits', './F560W/stage0/jw01040001001_03101_00004_mirimage_uncal.fits', './F560W/stage0/jw01040001001_03101_00005_mirimage_uncal.fits', './F560W/stage0/jw01040001001_0310j_00001_mirimage_uncal.fits', './F560W/stage0/jw01040001001_0310j_00002_mirimage_uncal.fits', './F560W/stage0/jw01040001001_0310j_00003_mirimage_uncal.fits', './F560W/stage0/jw01040001001_0310j_00004_mirimage_uncal.fits', './F560W/stage0/jw01040001001_0310j_00005_mirimage_uncal.fits', './F560W/stage0/jw01040001002_03101_00001_mirimage_uncal.fits', './F560W/stage0/jw01040001002_03101_00002_mirimage_uncal.fits', './F560W/stage0/jw01040001002_03101_00003_mirimage_uncal.fits', './F560W/stage0/jw01040001002_03101_00004_mirimage_uncal.fits', './F560W/stage0/jw01040001002_03101_00005_mirimage_uncal.fits', './F560W/stage0/jw01040001002_0310j_000

In [ ]:
if run_detector1:
    miri_detector1(miri_uncal_files, output_dir, linfile=linfile)

2022-07-15 09:05:16,694 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-07-15 09:05:16,696 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-07-15 09:05:16,699 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-07-15 09:05:16,701 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-07-15 09:05:16,703 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-07-15 09:05:16,705 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-07-15 09:05:16,707 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-07-15 09:05:16,709 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-07-15 09:05:16,711 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-07-15 09:05:16,713 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-07-15 0

./F560W/stage0/jw01040001001_03101_00001_mirimage_uncal.fits


2022-07-15 09:05:16,853 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': './F560W/stage1', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_calibrated_ramp': False, 'steps': {'group_scale': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'dq_init': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'saturation': {'pre_hooks': [], 'post_hooks': [], 'output_file': 

2022-07-15 09:05:21,535 - CRDS - INFO -  Fetching  /home/kgordon/crds_cache/mappings/jwst/jwst_miri_0301.imap    5.2 K bytes  (16 / 19 files) (88.9 K / 103.4 K bytes)
2022-07-15 09:05:21,639 - CRDS - INFO -  Fetching  /home/kgordon/crds_cache/mappings/jwst/jwst_fgs_apcorr_0003.rmap    4.1 K bytes  (17 / 19 files) (94.1 K / 103.4 K bytes)
2022-07-15 09:05:21,746 - CRDS - INFO -  Fetching  /home/kgordon/crds_cache/mappings/jwst/jwst_fgs_0103.imap    4.6 K bytes  (18 / 19 files) (98.2 K / 103.4 K bytes)
2022-07-15 09:05:21,854 - CRDS - INFO -  Fetching  /home/kgordon/crds_cache/mappings/jwst/jwst_0927.pmap        580 bytes  (19 / 19 files) (102.8 K / 103.4 K bytes)
2022-07-15 09:05:22,469 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/home/kgordon/crds_cache/references/jwst/miri/jwst_miri_dark_0082.fits'.
2022-07-15 09:05:22,470 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/home/kgordon/crds_cache/references/jwst/miri/jwst_miri_gain_0

2022-07-15 09:05:26,479 - stpipe.Detector1Pipeline.reset - INFO - Step reset done
2022-07-15 09:05:26,603 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(1, 6, 1024, 1032) from jw01040001001_03101_00001_mirimage_uncal.fits>,).
2022-07-15 09:05:26,606 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': './F560W/stage0'}
2022-07-15 09:05:26,659 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /home/kgordon/crds_cache/references/jwst/miri/jwst_miri_linearity_0032.fits
2022-07-15 09:05:27,200 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-07-15 09:05:27,335 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<R

...

2022-07-15 09:11:26,913 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 4
2022-07-15 09:11:26,916 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 1
2022-07-15 09:11:26,998 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 4
2022-07-15 09:11:27,001 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 1
2022-07-15 09:11:27,021 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 4
2022-07-15 09:11:27,025 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 1
2022-07-15 09:11:27,315 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-07-15 09:11:27,513 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<ImageModel(1024, 1032) from jw01040001003_03101_00003_mirimage_uncal.fits>,).
2022-07-15 09:11:27,517 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_

./F560W/stage0/jw01040001003_03101_00004_mirimage_uncal.fits


2022-07-15 09:11:28,481 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline running with args ('./F560W/stage0/jw01040001003_03101_00004_mirimage_uncal.fits',).
2022-07-15 09:11:28,505 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': './F560W/stage1', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_calibrated_ramp': False, 'steps': {'group_scale': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'dq_init': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': Tru

2022-07-15 09:11:29,845 - stpipe.Detector1Pipeline.group_scale - INFO - NFRAMES and FRMDIVSR are equal; correction not needed
2022-07-15 09:11:29,846 - stpipe.Detector1Pipeline.group_scale - INFO - Step will be skipped
2022-07-15 09:11:29,852 - stpipe.Detector1Pipeline.group_scale - INFO - Step group_scale done
2022-07-15 09:11:30,036 - stpipe.Detector1Pipeline.dq_init - INFO - Step dq_init running with args (<RampModel(1, 6, 1024, 1032) from jw01040001003_03101_00004_mirimage_uncal.fits>,).
2022-07-15 09:11:30,039 - stpipe.Detector1Pipeline.dq_init - INFO - Step dq_init parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': './F560W/stage0'}
2022-07-15 09:11:30,097 - stpipe.Detector1Pipeline.dq_init - INFO - Using MASK reference file /home/kgordon/crds_cache/references/jwst/miri/

2022-07-15 09:11:38,637 - stpipe.Detector1Pipeline.refpix - INFO - MIRI full frame data
2022-07-15 09:11:38,639 - stpipe.Detector1Pipeline.refpix - INFO - The following parameter is valid for this mode:
2022-07-15 09:11:38,641 - stpipe.Detector1Pipeline.refpix - INFO - odd_even_rows = True
2022-07-15 09:11:38,642 - stpipe.Detector1Pipeline.refpix - INFO - The following parameters are not applicable and are ignored:
2022-07-15 09:11:38,644 - stpipe.Detector1Pipeline.refpix - INFO - use_side_ref_pixels = False
2022-07-15 09:11:38,645 - stpipe.Detector1Pipeline.refpix - INFO - odd_even_columns = False
2022-07-15 09:11:38,646 - stpipe.Detector1Pipeline.refpix - INFO - side_smoothing_length = False
2022-07-15 09:11:38,647 - stpipe.Detector1Pipeline.refpix - INFO - side_gain = False
2022-07-15 09:11:38,649 - stpipe.Detector1Pipeline.refpix - INFO - Subtracting initial read from each integration


In [ ]:
# fix the making of the rate images from the rateints images
#   needed for multiple integrations in regions near saturation
miri_rate_files = glob.glob(f"./{filter}/stage1/*ge_rate.fits")
for cfile in miri_rate_files:
    fix_rateints_to_rate(cfile)

In [ ]:
# look at one of the images
miri_rate_files = glob.glob(f"./{filter}/stage1/*ge_fixed_rate.fits")
miri_rate = datamodels.open(miri_rate_files[0])
drange = drange_raw

In [ ]:
show_image(miri_rate.data, vmin=drange[0], vmax=drange[1], dmap=dmap)

In [ ]:
# display the images
fig = plt.figure(1, figsize=(12, 12))
ncols = len(miri_rate_files) // 3
grid = ImageGrid(fig, 111, nrows_ncols=(ncols+1, 3), axes_pad=0)

for i, cfile in enumerate(miri_rate_files):
    with datamodels.open(cfile) as miri_rate:
        cimage = miri_rate.data
        norm_data = simple_norm(cimage, "sqrt", min_cut=drange[0], max_cut=drange[1])
        grid[i].imshow(cimage, norm=norm_data, origin="lower", cmap=plt.get_cmap(dmap))
        grid[i].axis('off')
        grid[i].set_xticks([])
        grid[i].set_yticks([])

## CALWEBB_IMAGE2

In [ ]:
miri_rate_files = glob.glob(f"./{filter}/stage1/*ge_fixed_rate.fits")
output_dir = f'./{filter}/stage2'
print(miri_rate_files)

In [ ]:
if run_image2:
    miri_image2(miri_rate_files, output_dir, flatfile=flatfile)

In [ ]:
drange = drange_cal
# look at one of the images
miri_cal_files = glob.glob(f"./{filter}/stage2/*ge_fixed_cal.fits")
miri_cal = datamodels.open(miri_cal_files[0])
show_image(miri_cal.data, vmin=drange[0], vmax=drange[1], dmap=dmap)

In [ ]:
show_image(miri_cal.dq, 0, 10, dmap=dmap)

In [ ]:
# display the images
fig = plt.figure(1, figsize=(12, 12))
ncols = len(miri_cal_files) // 3
grid = ImageGrid(fig, 111, nrows_ncols=(ncols+1, 3), axes_pad=0)

for i, cfile in enumerate(miri_cal_files):
    with datamodels.open(cfile) as miri_cal:
        cimage = miri_cal.data
        norm_data = simple_norm(cimage, "sqrt", min_cut=drange[0], max_cut=drange[1])
        grid[i].imshow(cimage, norm=norm_data, origin="lower", cmap=plt.get_cmap(dmap))
        grid[i].axis('off')
        grid[i].set_xticks([])
        grid[i].set_yticks([])

## CALWEBB_IMAGE3

In [ ]:
# update the WCS for mosaic tile 1
miri_cal_files = glob.glob(f"./{filter}/stage2/{basename}1*ge_fixed_cal.fits")
for cfile in miri_cal_files:
    shift_cal_wcs(cfile, tile1_shifts)

# update the WCS for mosaic tile 2
miri_cal_files = glob.glob(f"./{filter}/stage2/{basename}2*ge_fixed_cal.fits")
for cfile in miri_cal_files:
    shift_cal_wcs(cfile, tile2_shifts)
    
# update the WCS for mosaic tile 3
miri_cal_files = glob.glob(f"./{filter}/stage2/{basename}3*ge_fixed_cal.fits")
for cfile in miri_cal_files:
    shift_cal_wcs(cfile, tile3_shifts)
    
# update the WCS for mosaic tile 4
miri_cal_files = glob.glob(f"./{filter}/stage2/{basename}4*ge_fixed_cal.fits")
for cfile in miri_cal_files:
    shift_cal_wcs(cfile, tile4_shifts)
    
# update the WCS for mosaic tile 5
miri_cal_files = glob.glob(f"./{filter}/stage2/{basename}5*ge_fixed_cal.fits")
for cfile in miri_cal_files:
    shift_cal_wcs(cfile, tile5_shifts)

In [ ]:
miri_cal_files = glob.glob(f"./{filter}/stage2/j*ge_fixed_wcs_cal.fits")
output_dir = f"./{filter}/stage3/"
print(miri_cal_files)

if rotation is not None:
    miri_asn_name = f'miri_{filter}_stage3_fixedrot_asn'
else: 
    miri_asn_name = f'miri_{filter}_stage3_asn'
miri_asn = asn_from_list.asn_from_list(miri_cal_files, rule=DMS_Level3_Base, product_name=miri_asn_name)

miri_asn_file = f'{miri_asn_name}.json'
with open(miri_asn_file, 'w') as outfile:
    name, serialized = miri_asn.dump(format='json')
    outfile.write(serialized)
    
print(miri_asn_file)

In [ ]:
if run_image3:
    miri_image3(miri_asn_file, output_dir, fwhm=fwhm, matchbkg=matchbkg, 
                crval=crval, rotation=rotation, 
                tweakreg=tweakreg, align_to_gaia=align_to_gaia)

In [ ]:
# Look at the resulting mosaic image
miri_mosaic_file = output_dir + miri_asn_name + '_i2d.fits'
miri_catalog_file = output_dir + miri_asn_name + '_cat.ecsv'

In [ ]:
# Open the mosaic image
miri_mosaic = datamodels.open(miri_mosaic_file)

In [ ]:
# Read in the source catalog
miri_source_cat = ascii.read(miri_catalog_file)

In [ ]:
# Look at the mosaic
show_image(miri_mosaic.data, vmin=drange[0], vmax=drange[1], dmap=dmap)

In [ ]:
# Look at the mosaic data read noise uncertainties (i.e., coverage map)
show_image(miri_mosaic.var_rnoise, vmin=0.0, vmax=0.01, dmap=dmap)

In [ ]:
# Look at the mosaic data poisson uncertainties
show_image(miri_mosaic.var_poisson, vmin=0, vmax=0.01, dmap=dmap)

In [ ]:
# Look at the mosaic total data uncertainties
show_image(miri_mosaic.err, vmin=0, vmax=0.2, dmap=dmap)

In [ ]:
# Show the catalog sources on the mosaic
overlay_catalog(miri_mosaic.data, miri_source_cat, flux_limit=5e-7, vmin=drange[0], vmax=drange[1],
                title='Final MIRI mosaic with source catalog', dmap=dmap)